In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# xlrd is required to read Excel files
# pip install xlrd>=2.0.1

In [4]:
filename = '../data/titanic3.xls'

df = pd.read_excel(filename)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Beyond 1

Create a series (`most_common_destinations`) in which the index contains the unique values from the `embarked` column, and the values are the most common destination for each value of `embarked`.

In [5]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'home.dest'].value_counts().index[0]
    
most_common_destinations

S           New York, NY
C           New York, NY
Q    Ireland Chicago, IL
dtype: object

`home.dest` 특성에 포함된 값들이 반드시 거주지/목적지 형식을 갖지는 않음. 

In [6]:
df['home.dest'].unique()[:5]

array(['St Louis, MO', 'Montreal, PQ / Chesterville, ON', 'New York, NY',
       'Hudson, NY', 'Belfast, NI'], dtype=object)

`'St Louis, MO'`의 경우처럼 주어진 값이 거주지인지 목적지인지 데이터만으로는 판단할 수 없다.
그런 경우엔 필요에 따라 추가 조사를 통해 확인해야한다.

아래 코드는 목적지가 명확한 데이터만 따로 추출하여 `'dest'` 특성으로 추가한다.

In [7]:
df['dest'] = df['home.dest'].apply(
        lambda x: x.split('/', 1)[1].strip() if isinstance(x, str) and '/' in x else np.nan
    )

위 코드 대신 아래 코드 사용 가능.
단 시리즈 객체의 `str` 속성은 문자열로 변환된 값들로 구성된, 시리즈와 유사한 객체를
가리키며, 동시에 문자열을 다루는 다양한 문자열 메서드를 제공한다.

In [8]:
df['dest'] = df['home.dest'].str.split('/').str[1].str.strip()

목적지가 명시된 데이터는 총 171개에 불과하다.

In [9]:
df['dest'].notnull().sum()

np.int64(171)

새로 생성된 `'dest'` 열의 정보만을 이용할 경우 승선항구별 목적지의 최빈값이 달라진다.

In [10]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'dest'].value_counts().index[0]
    
most_common_destinations

S          Akron, OH
C    Cooperstown, NY
Q         Bangor, ME
dtype: object

하지만 위 결과의 신뢰도는 아무도 보장할 수 없음에 주의한다.
이유는 `'dest'` 열을 새로 생성하는 과정에서 `'주거지/목적지'` 형식이 아닌 데이터는 제거되었기 때문이다.

그렇다고 아래 코드에서처럼 주거지 또는 목적지 하나만 있는 경우를 모두 목적지로 간주하는 것 또한 바람직하지 않다.
이유는 `'St Louis, MO'`의 경우처럼 주어진 값이 거주지인지 목적지인지 데이터만으로는 판단할 수 없기 때문이다.

In [11]:
df['dest'] = df['home.dest'].apply(
        lambda x: x.split('/', 1)[1].strip() if isinstance(x, str) and '/' in x else x
    )

또는

In [12]:
df['dest'] = df['home.dest'].str.split('/').str[-1].str.strip()

In [13]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'dest'].value_counts().index[0]
    
most_common_destinations

S           New York, NY
C           New York, NY
Q    Ireland Chicago, IL
dtype: object

# Beyond 2

Now replace `NaN` values in the `home.dest` coumn with values from `embarked`.  

먼저 `'home.dest'` 열에 결측치가 있는 데이터를 확인한다.

In [14]:
df[df['home.dest'].isna()]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,dest
13,1,1,"Barber, Miss. Ellen ""Nellie""",female,26.0,0,0,19877,78.8500,NaN,S,6,NaN,NaN,NaN
18,1,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C,8,NaN,NaN,NaN
23,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,NaN,C,4,NaN,NaN,NaN
24,1,1,"Bird, Miss. Ellen",female,29.0,0,0,PC 17483,221.7792,C97,S,8,NaN,NaN,NaN
28,1,1,"Bissette, Miss. Amelia",female,35.0,0,0,PC 17760,135.6333,C99,S,8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN,NaN


`'home.dest'` 열의 결측치를 `'embarked'` 열의 값으로 채운다.

In [15]:
df['home.dest'] = df['home.dest'].fillna(df['embarked'])

`'home.dest'`에 여전히 남은 결측치가 남아 있다.

In [16]:
df[df['home.dest'].isna()]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,dest
168,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,6,NaN,NaN,NaN


# Beyond 3

Now use the `most_common_destinations` series to replace values in `home.dest` with the most common values for each embarkation point.

In [17]:
df['home.dest'] = df['home.dest'].replace(most_common_destinations)